In [8]:
import psycopg2
import csv
import io
import hashlib

In [6]:
# Connection parameters - replace these with your database credentials
conn_params = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'postgrespw',
    'host': 'localhost',
    'port': '55000',
    'options': '-c search_path=financial_data',
}

# Create a connection
connection = psycopg2.connect(**conn_params)

In [14]:
csv_file_path = 'domain/db/data/Chase4091CreditCard.CSV'
def generate_checksum(date, amount, description):
    checksum = hashlib.sha256(f"{date}{amount}{description}".encode()).hexdigest()
    return checksum

def populate_db_with_csv_data():
    cursor = connection.cursor()
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Generate the checksum ID
            transaction_id = generate_checksum(row['Transaction Date'], row['Amount'], row['Description'])
    
            # Create the SQL insert statement
            sql = """
            INSERT INTO Transaction (id, date, amount, description, category, type, account_id) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING;  -- This will ignore the insertion if the transaction ID already exists
            """
            
            # Execute the SQL statement
            try:
                cursor.execute(sql, (
                    transaction_id,
                    row['Transaction Date'],
                    row['Amount'],
                    row['Description'],
                    row['Category'],
                    row['Type'],
                    '1'  # The account_id is set to '1' for all transactions
                ))
            except psycopg2.Error as e:
                print(f"Error inserting transaction with ID {transaction_id}: {e}")
                connection.rollback()  # Rollback the transaction on error
            else:
                connection.commit()  # Commit the transaction if successful
    
        # Close the database connection
        cursor.close()

populate_db_with_csv_data()        

In [5]:
def query_to_csv_string(connection, query):
    # Cursor for database connection
    cursor = connection.cursor()
    # Execute the query
    cursor.execute(query)
    # Fetch the results
    results = cursor.fetchall()
    # Column names from the cursor
    col_names = [desc[0] for desc in cursor.description]
    # Create a CSV output from the results
    output = io.StringIO()
    csv_writer = csv.writer(output)
    # Write the header
    csv_writer.writerow(col_names)
    # Write the data
    csv_writer.writerows(results)
    # Get the CSV string
    csv_string = output.getvalue()
    output.close()
    return csv_string
# SQL Query
query = "SELECT * FROM account;"  # Replace with your query
# Get the CSV string
csv_string = query_to_csv_string(connection, query)
# Print or use the CSV string
print(csv_string)

id,account_type,account_name
